In [15]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='normalized')
# print(df)

In [18]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(3)

#emotion arr
# emotions = ["amazement", "calmness", "power", "joyful activation", "sadness"]
emotions = ["solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]

In [19]:
train_percentage = 0.8

train_song = []
test_song = []
train_emotion = []
test_emotion = []

row_indexes = np.arange(1,226)
np.random.shuffle(row_indexes)

def get_data(indexes):
    print(indexes)
    song = []
    emotion = []
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:
        row = torch.tensor([sheet.cell_value(x, j) for j in range(3, 11)], device=device).float()
        trackid = int(sheet.cell_value(x, 0))
        song.append(dataset.joinpath("{}.mp3".format(trackid)))
        emotion.append(row)
            
    return song, emotion
    
song, emotion = get_data(row_indexes)
print(song)
print(emotion)
# test_song, test_emotion = get_data(test_indexes)

train_song = [song[i] for i in range(len(song)) if i < len(song)*train_percentage]
test_song = [song[i] for i in range(len(song)) if i >= len(song)*train_percentage]

train_emotion = [emotion[i] for i in range(len(emotion)) if i < len(emotion)*train_percentage]
test_emotion = [emotion[i] for i in range(len(emotion)) if i >= len(emotion)*train_percentage]

# num_maxes = [0 for _ in emotions]

# for row in emotion:
#     i = torch.argmax(row)
#     print(i)
#     num_maxes[i] += 1
    
# print(num_maxes)

[ 69  42 151 148 152 115  52  29 168 158  76  21 209 221 112 223 176 131
 116  91  28 172 114  81  44  48 106  38  33  55 120   7  83  14 154 220
 193 117  75 202  60 215 205  97 170 164 107  63  39 167 171 169 103  50
  82 129 104   3 204  26 212  96 163  32  78 194  56  87  30 139 213  80
  36 138 191  46 126 130  88  99  53 196 225  18  85  41 216   5 102 173
  37 207  90  62 155  57  77   6  94  17 210  23 128 182 110 200 218 150
  89 165   8  79 199  10 222 162  58  35 179  86 188   2 143  68 186  73
 123  12 160  15  92 197  54  43 201  13 211 144  22 147  24  74 125 137
  64 122 156 206  61  72  71 153 140 135   1 184 198 111 127  45 134 185
 180  49  67 121 142 187 132 208   9 108 141 174  66  65  16 217 161 136
 224 203 149 177 100  11 178 183 105 101  84  98 192 214 159  95  31 219
  27 133  47  19 166  51 145 195 146 118  34  40 157 119 190   4 124  70
 189 181  93  25 109 113 175  20  59]
[PosixPath('/home/czf/memories/SongEmotionModel/SongEmotionDataset/104.mp3'), PosixPat

[tensor([10., 13., 21., 11.,  2.,  6.,  7.,  9.], device='cuda:0'), tensor([3., 6., 6., 6., 1., 2., 1., 2.], device='cuda:0'), tensor([ 9.,  3.,  6.,  6.,  5., 41.,  2.,  0.], device='cuda:0'), tensor([ 4.,  7.,  4.,  5., 10., 38.,  7.,  1.], device='cuda:0'), tensor([22.,  2.,  4.,  2., 17., 28.,  5.,  4.], device='cuda:0'), tensor([4., 5., 7., 9., 0., 0., 0., 6.], device='cuda:0'), tensor([2., 6., 4., 3., 1., 2., 1., 2.], device='cuda:0'), tensor([6., 3., 4., 6., 0., 2., 4., 2.], device='cuda:0'), tensor([ 1.,  0.,  0.,  0.,  5., 12.,  5.,  0.], device='cuda:0'), tensor([ 5.,  0.,  0.,  0.,  9., 12.,  3.,  0.], device='cuda:0'), tensor([1., 2., 7., 6., 2., 4., 2., 4.], device='cuda:0'), tensor([4., 0., 3., 3., 3., 2., 4., 0.], device='cuda:0'), tensor([ 7.,  3.,  5.,  6.,  1.,  0.,  1., 10.], device='cuda:0'), tensor([10.,  7., 19., 15.,  8.,  4.,  6., 20.], device='cuda:0'), tensor([ 4.,  5.,  8., 10.,  1.,  1.,  2.,  4.], device='cuda:0'), tensor([3., 5., 3., 2., 1., 1., 1., 6.], d

In [20]:
# train_emotion

In [21]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        self.cache = {}
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index not in self.cache.keys():
#             print("index of " + str(index) + " was cached!")
            data, rate = librosa.load(self.mp3[index], sr=16000, duration=10)
            mfccs = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40)
            assert rate == 16000
            sample_tensor = torch.tensor(mfccs, device=device).float()
            downsampled_tensor = sample_tensor #[::10]
#             print(mfccs.shape, data.shape)

            self.cache[index] = (downsampled_tensor, F.softmax(self.labels[index]))
#         else:
#             print("index was cached! index of " + str(index))
        return self.cache[index]

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (2,2))
        self.conv2 = nn.Conv2d(32, 64, (2,2))
        self.pool1 = nn.MaxPool2d((2,2))
        self.drop1 = nn.Dropout(p=0.50)
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(188480, 128)
        self.drop2 = nn.Dropout(p=0.5)
        self.dense2 = nn.Linear(128, 8)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = F.relu(x)
        
        x = self.drop2(x)
        x = self.dense2(x)
        x = F.relu(x)

        return F.log_softmax(x, dim = 1)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (flat1): Flatten()
  (dense1): Linear(in_features=188480, out_features=128, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=128, out_features=8, bias=True)
)


In [28]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 180
Test set size: 45


In [29]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)

        kl = nn.KLDivLoss()
        loss = kl(output, target)
        
        loss.backward()
        optimizer.step()
        
#         print(output)
#         print(target)
#         print("\n")
        
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [30]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data. unsqueeze_(1)
        output = model(data)
        print(output)
#         output = output.permute(1, 0, 2)
        pred = output.max(1)[1] # get the index of the max log-probability 
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [31]:
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 100):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/180 (0%)]	Loss: 0.620333
Train Epoch: 1 [40/180 (22%)]	Loss: 0.176380
Train Epoch: 1 [80/180 (43%)]	Loss: 0.212933
Train Epoch: 1 [120/180 (65%)]	Loss: 0.187738
Train Epoch: 1 [160/180 (87%)]	Loss: 0.200677
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2

Train Epoch: 3 [120/180 (65%)]	Loss: 0.148960
Train Epoch: 3 [160/180 (87%)]	Loss: 0.184734
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.079

Train Epoch: 5 [40/180 (22%)]	Loss: 0.189365
Train Epoch: 5 [80/180 (43%)]	Loss: 0.204374
Train Epoch: 5 [120/180 (65%)]	Loss: 0.206989
Train Epoch: 5 [160/180 (87%)]	Loss: 0.210378
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794

Train Epoch: 7 [80/180 (43%)]	Loss: 0.192158
Train Epoch: 7 [120/180 (65%)]	Loss: 0.192627
Train Epoch: 7 [160/180 (87%)]	Loss: 0.186616
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794

Train Epoch: 9 [120/180 (65%)]	Loss: 0.202776
Train Epoch: 9 [160/180 (87%)]	Loss: 0.206285
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.079

...

Train Epoch: 91 [120/180 (65%)]	Loss: 0.176980
Train Epoch: 91 [160/180 (87%)]	Loss: 0.193899
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9

Train Epoch: 93 [40/180 (22%)]	Loss: 0.174465
Train Epoch: 93 [80/180 (43%)]	Loss: 0.167187
Train Epoch: 93 [120/180 (65%)]	Loss: 0.168216
Train Epoch: 93 [160/180 (87%)]	Loss: 0.194990
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.

Train Epoch: 95 [80/180 (43%)]	Loss: 0.219312
Train Epoch: 95 [120/180 (65%)]	Loss: 0.173100
Train Epoch: 95 [160/180 (87%)]	Loss: 0.183154
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0

Train Epoch: 97 [120/180 (65%)]	Loss: 0.194777
Train Epoch: 97 [160/180 (87%)]	Loss: 0.220104
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9

Train Epoch: 99 [40/180 (22%)]	Loss: 0.146707
Train Epoch: 99 [80/180 (43%)]	Loss: 0.195501
Train Epoch: 99 [120/180 (65%)]	Loss: 0.216940
Train Epoch: 99 [160/180 (87%)]	Loss: 0.172733
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.0974, -2.0974, -1.9620, -2.0974, -2.0974, -2.0974, -2.0974, -2.0974],
        [-2.0974, -2.

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

## NOTES

below is the mfccs notes / random code

In [13]:
audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# [print(x) for x in audio]

#convert audio into 2d array
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccsscaled = np.mean(mfccs.T,axis=0)
print(mfccs.shape, audio.shape)
mfccs

/home/czf/memories/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(40, 2586) (1323648,)


array([[-5.30341797e+02, -4.07741577e+02, -3.27536621e+02, ...,
        -2.39811523e+02, -1.96744080e+02, -1.44711777e+02],
       [ 5.81265569e-01,  1.03006027e+02,  1.29354553e+02, ...,
         1.48707626e+02,  1.45873001e+02,  1.28202530e+02],
       [ 4.58764762e-01,  7.53921986e+00, -1.18814125e+01, ...,
        -2.51551704e+01, -1.92207527e+01, -1.79366188e+01],
       ...,
       [ 3.11299562e-01, -1.29907084e+00,  1.18818974e+00, ...,
        -6.58579540e+00, -3.34302998e+00, -4.75482178e+00],
       [ 2.23848164e-01, -3.19489312e+00, -2.78556681e+00, ...,
        -1.36089420e+01, -6.40699673e+00, -5.27228928e+00],
       [ 8.67742151e-02,  1.31472754e+00, -1.41885233e+00, ...,
         3.34440261e-01,  1.14392626e+00, -3.62402201e-02]], dtype=float32)

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    

In [10]:
# train_percentage = 0.8

# min_train = min_total*train_percentage
# min_test = min_total - min_train

# train_totals = torch.zeros(len(emotions))

# while 


# for i in range(1, 401):
#     count_total = sheet.cell_value(i, 7)
    
#     emotions_counter = [0 for e in emotions]
#     if i % 5 == 0:
#         test_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(5):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         test_emotion.append(torch.tensor(emotion_arr, device=device).float())
        
#     emotions_counter = [0 for e in emotions]
#     else:
#         train_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(5):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         train_emotion.append(torch.tensor(emotion_arr, device=device))

# print(len(train_song), len(test_song))
# print(len(train_emotion), len(test_emotion))

In [ ]:
# def train(model, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         print(data)
#         optimizer.zero_grad()
#         data.unsqueeze_(1)
#         data = data.requires_grad_() #set requires_grad to True for training
#         output = model(data)
# #         output = output.view(-1, len(emotions))
# #         print(output.shape, target.shape)
# #         print(output, target)
# #         loss = F.kl_div(output, target)
#         kl = nn.KLDivLoss()
#         loss = kl(output, target)
# #         loss = F.cross_entropy(output, target)
# #         loss = nn.CrossEntropyLoss(output, target)
# #         loss = F.nll_loss(output, target)
#         loss.backward()
#         print(loss)
#         optimizer.step()
#         print(output)
#         print(target)
#         print("\n")
# #         scheduler.step()
#         if batch_idx % log_interval == 0: #print training stats
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss))

In [15]:
train_percentage = 0.8
allowed_exceedance = 0

train_song = []
test_song = []
train_emotion = []
test_emotion = []

row_indexes = np.arange(1,401)
np.random.shuffle(row_indexes)

# train_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i < len(row_indexes)*train_percentage]
# test_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i >= len(row_indexes)*train_percentage]

def get_data(indexes):
    song = []
    emotion = []
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:    
        row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(5)], device=device).float()
        totals += F.softmax(row)

    min_total = torch.min(totals)
    print(totals)
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:
        row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(5)], device=device).float()
        
        if torch.max(totals + row) < min_total*(1 + allowed_exceedance):
            song.append(dataset.joinpath("{}.mp3".format(x)))
            emotion.append(row)
            totals += F.softmax(row)
            
    print(totals)
    return song, emotion
    
song, emotion = get_data(row_indexes)
# test_song, test_emotion = get_data(test_indexes)

train_song = [song[i] for i in range(len(song)) if i < len(song)*train_percentage]
test_song = [song[i] for i in range(len(song)) if i >= len(song)*train_percentage]

train_emotion = [emotion[i] for i in range(len(emotion)) if i < len(emotion)*train_percentage]
test_emotion = [emotion[i] for i in range(len(emotion)) if i >= len(emotion)*train_percentage]

num_maxes = [0 for _ in emotions]

for row in emotion:
    i = torch.argmax(row)
    num_maxes[i] += 1
    
print(num_maxes)

tensor([ 11.4343, 157.9053,  55.5535, 120.1634,  54.9434], device='cuda:0')
tensor([3.1017, 9.4848, 9.0917, 8.7741, 8.5477], device='cuda:0')
[1, 9, 11, 10, 8]


/home/armaan.goel/memories/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/armaan.goel/memories/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
